In [29]:
import numpy as np
import pandas as pd
import random
from os.path import join
import os
import pickle
from glob import glob
import json
import sys

sys.path.append('../annotations/')
from color_harmony import extract_harmonies

sys.path.append('../disentanglement/')
from disentanglement import DisentanglementBase

sys.path.append('../stylegan')
from networks_stylegan3 import *
import dnnlib 
import legacy


In [33]:
DATA_DIR = '../data/'
df_scores_vectors = glob(DATA_DIR + 'scores_modifications*')
df_scores = pd.DataFrame()
for df_score in df_scores_vectors:
    df_score_l = pd.read_csv(df_score)
    df_scores = pd.concat([df_scores, df_score_l], axis=0)

df_scores = df_scores.sort_values('Final Score', ascending=False).reset_index()
df_scores.loc[df_scores['Variable'] == 'V1', 'Feature'] = 'V1'
df_scores.loc[df_scores['Variable'] == 'S1', 'Feature'] = 'S1'
df_scores['Method General'] = df_scores['Method'].str.split('_').apply(lambda x: x[0].replace('False', '').replace('True', ''))
# df_scores_inter = df_scores[df_scores['Method'] == 'InterfaceGAN_LR_1_8_Color_True']
# df_scores_sv = df_scores[df_scores['Method'] == 'ShapleyVec_True_0.5_8_Color_LR']
# df_scores_ss = df_scores[df_scores['Method'] == 'StyleSpaceTrue_20_8_Color_False']

# df_scores = pd.concat([df_scores_inter, df_scores_sv, df_scores_ss], axis=0)

df_scores = df_scores.groupby(['Method General', 'Feature']).first()
df_scores = df_scores.reset_index()

modifications_vectors = glob(DATA_DIR + 'modifications*')
df_modifications_vectors = pd.DataFrame()
for df_modification in modifications_vectors:
    df_modification_l = pd.read_csv(df_modification)
    df_modifications_vectors = pd.concat([df_modifications_vectors, df_modification_l], axis=0)
df_modifications_vectors.loc[df_modifications_vectors['Variable'] == 'V1', 'Feature'] = 'V1'
df_modifications_vectors.loc[df_modifications_vectors['Variable'] == 'S1', 'Feature'] = 'S1'

separation_vectors = glob(DATA_DIR + '*_separation_vector*.csv')
df_sep_vecs = pd.DataFrame()
for df_sep_vec in separation_vectors:
    if 'modifications' not in df_sep_vec:
        df_sep_vec_l = pd.read_csv(df_sep_vec)
        df_sep_vecs = pd.concat([df_sep_vecs, df_sep_vec_l], axis=0)
df_sep_vecs.loc[df_sep_vecs['Variable'] == 'V1', 'Feature'] = 'V1'
df_sep_vecs.loc[df_sep_vecs['Variable'] == 'S1', 'Feature'] = 'S1'

df_scores = df_scores.merge(df_sep_vecs, left_on=['Feature','Variable','Method'], right_on=['Feature','Variable','Method'], how='left')
df_scores = df_scores.merge(df_modifications_vectors, left_on=['Feature','Variable','Method'], right_on=['Feature','Variable','Method'], how='left')
df_scores['Separation Vector (np)'] = [np.array([float(x.strip('[] ')) for x in row['Separation Vector'].replace('\n', ' ').split(' ') if x.strip('[] ') != '']) for i, row in df_scores.iterrows()]
df_scores.head()

Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.


,Method General,Feature,index,Method,Variable,Broad,Optimal lambda,Total Score,Score lambda 1,Score lambda 2,...,Monochromatic,Analogous,Complementary,Triadic,Split Complementary,Double Complementary,Color,Space_y,Subfolder_y,Separation Vector (np)
0,InterfaceGAN,BW,33,InterfaceGAN_LR_1_8_Color_True,Color,1.0,7.0,0.661,0.108,0.189,...,0.0,0.0,0.0,0.0,0.0,0.0,Red,w,interfaceGAN/color/,"[-0.0266842892, -0.00821213069, -0.00307416816..."
1,InterfaceGAN,BW,33,InterfaceGAN_LR_1_8_Color_True,Color,1.0,7.0,0.661,0.108,0.189,...,0.0,0.0,0.0,0.0,0.0,0.0,BW,w,interfaceGAN/color/,"[-0.0266842892, -0.00821213069, -0.00307416816..."
2,InterfaceGAN,BW,33,InterfaceGAN_LR_1_8_Color_True,Color,1.0,7.0,0.661,0.108,0.189,...,0.0,1.0,0.0,0.0,0.0,0.0,BW,w,interfaceGAN/color/,"[-0.0266842892, -0.00821213069, -0.00307416816..."
3,InterfaceGAN,BW,33,InterfaceGAN_LR_1_8_Color_True,Color,1.0,7.0,0.661,0.108,0.189,...,0.0,1.0,0.0,0.0,0.0,0.0,BW,w,interfaceGAN/color/,"[-0.0266842892, -0.00821213069, -0.00307416816..."
4,InterfaceGAN,BW,33,InterfaceGAN_LR_1_8_Color_True,Color,1.0,7.0,0.661,0.108,0.189,...,0.0,1.0,0.0,0.0,0.0,0.0,BW,w,interfaceGAN/color/,"[-0.0266842892, -0.00821213069, -0.00307416816..."


In [34]:
df_scores['Feature'].value_counts()

BW         4800
Blue       4800
Brown      4800
Cyan       4800
Green      4800
Magenta    4800
Red        4800
Yellow     4800
S1         1600
V1         1600
Name: Feature, dtype: int64

In [32]:
model_file = '../data/network-snapshot-005000.pkl'
with dnnlib.util.open_url(model_file) as f:
        model = legacy.load_network_pkl(f)['G_ema'] # type: ignore
        
annotations_file = '../data/seeds0000-100000.pkl'
with open(annotations_file, 'rb') as f:
        annotations = pickle.load(f)

In [35]:
disent = DisentanglementBase(model, annotations, df_scores, 'w', None, None,)

Using device cpu


In [36]:
idx = 119
vector = np.zeros(512)
vector[idx] = 1

vector = df_scores[df_scores['Method'] == 'InterfaceGAN_LR_1_8_Color_True'][df_scores['Feature'] == 'Brown']['Separation Vector (np)'].values[0]
vector

Boolean Series key will be reindexed to match DataFrame index.


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
seed = random.randint(0,10000)
seed = 1013
print(seed)

1013


In [12]:
disent.generate_changes(seed, vector, min_epsilon=0, max_epsilon=30, count=7, savefig=True, subfolder='custom', feature='Color', method=f'Custom{idx}')

([<PIL.Image.Image image mode=RGB size=256x256>,
  <PIL.Image.Image image mode=RGB size=256x256>,
  <PIL.Image.Image image mode=RGB size=256x256>,
  <PIL.Image.Image image mode=RGB size=256x256>,
  <PIL.Image.Image image mode=RGB size=256x256>,
  <PIL.Image.Image image mode=RGB size=256x256>,
  <PIL.Image.Image image mode=RGB size=256x256>],
 array([ 0.,  5., 10., 15., 20., 25., 30.]))

In [37]:
example_seeds = [40851, 11703, 52921, 17639, 101, 99423, 70]

In [ ]:
# df_scores = df_scores.drop('level_0', axis=1)

In [38]:
for seed in example_seeds:
    print(seed)
    for idx in df_scores['Method'].unique():
        print(idx)
        for color in df_scores[df_scores['Method'] == idx]['Feature'].unique():
            print(color)
            filt = df_scores[df_scores['Method'] == idx][df_scores['Feature'] == color].reset_index()
            print(filt.shape)
            optimal_lambda = filt['Optimal lambda'][0]
            vec = filt['Separation Vector (np)'][0]
            disent.generate_changes(seed, vec, min_epsilon=0, max_epsilon=optimal_lambda*2, count=4, savefig=True, subfolder='custom', feature=f'Color {color}', method=f'Custom_{idx}', save_separately=True)

40851
InterfaceGAN_LR_1_8_Color_True
BW
(1600, 59)


Boolean Series key will be reindexed to match DataFrame index.


Cyan
(1600, 59)


Boolean Series key will be reindexed to match DataFrame index.


Magenta
(1600, 59)


...

Cyan
(1600, 59)


Boolean Series key will be reindexed to match DataFrame index.


Green
(1600, 59)


Boolean Series key will be reindexed to match DataFrame index.


KeyboardInterrupt: 